# Recurrent Neural Network in 2D and 3D lightcones

There are several ideas we would like to explore, concerning RNN networks.

1. Compessing spacial data to N parameters with CNN, with network weights independent of time (redshift) and feeding those to RNN network which in every step tries to predict astrophysical parameters, and then weight somehow the sum all of those in the loss function.
2. Using the whole CNN as a part of RNN network which updates it's weights in each iteration
3. Some hybrid of usual convolution in spacial dimensions and WaveNet-like convolutions in time dimension

In 1. and 2. for RNN part, we use LSTM cells

In [1]:
import numpy as np
import sys
import os
import json
import tensorflow as tf
# os.environ['KERAS_BACKEND'] = 'tensorflow'


# # #setting up GPU
# config = tf.compat.v1.ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 1. #setting the percentage of GPU usage
# config.gpu_options.visible_device_list = "0" #for picking only some devices
# # config.gpu_options.allow_growth = True

# #passing tf session to keras!
# # from keras.backend.tensorflow_backend import set_session
# tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config)) 

# # from keras import backend as K
tf.keras.backend.set_image_data_format('channels_last')

In [2]:
#working with sliced data shape (10000, 4, 25, 25, ~500)
DataLoc = "../data/"
XName = "data3D_boxcar444_sliced22_float32.npy"
YName = "databaseParams_float32.npy"
YbackupName = "databaseParams_min_max.txt"

In [57]:
X = {}
Y = {}
for sort in ["train", "val", "test"]:
    X[sort] = np.load(f"{DataLoc}{sort}/X_{XName}")
    X[sort] = X[sort][..., np.newaxis]
    Y[sort] = np.load(f"{DataLoc}{sort}/Y_{XName}")
#     Y[sort] = Y[sort][..., np.newaxis]

MemoryError: Unable to allocate array with shape (40000, 526, 25, 25) and data type float32

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, TimeDistributed, Dense
from tensorflow.keras.layers import Conv3D, Conv2D, Conv1D
from tensorflow.keras.layers import ConvLSTM2D
from tensorflow.keras.layers import BatchNormalization, Flatten

In [4]:
class HyperParams1:
    def __init__(
        self,
        LossDecay = 0.9,
        LossLength = 10,
        SummaryDimension = 5,
        ):
        self.LossDecay = LossDecay
        self.LossLength = LossLength
        self.SummaryDimension = SummaryDimension

In [5]:
HP = HyperParams1()
# input_shape = X["train"].shape[1:]
input_shape = (526, 25, 25, 1)
model1 = Sequential()
model1.add(TimeDistributed(Conv2D(filters=32, kernel_size=(5, 5), activation='relu'), input_shape = input_shape))
for i in range(5):
    model1.add(TimeDistributed(Conv2D(filters=64, kernel_size=(5, 5), activation='relu')))
model1.add(TimeDistributed(Flatten()))
model1.add(TimeDistributed(Dense(HP.SummaryDimension)))
model1.add(LSTM(16, return_sequences=True))
model1.add(LSTM(16, return_sequences=True))
model1.add(TimeDistributed(Dense(4)))
model1.summary()

InvalidArgumentError: Invalid device ordinal value (1). Valid range is [0, 0].
	while setting up XLA_GPU_JIT device number 1